In [1]:
import torch
import numpy as np

/home/jd/anaconda3/envs/marmot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
def beam_projection(pos, az):
    pos_angle = np.arctan2(pos[1],pos[0])
    # make sure pos_angle in range [0, 2pi)

    while pos_angle < 0:
        pos_angle += 2*np.pi
    while pos_angle > 2*np.pi:
        pos_angle -= 2*np.pi

    while pos_angle < 0:
        pos_angle += 2*np.pi
    while pos_angle > 2*np.pi:
        pos_angle -= 2*np.pi
    
    print("mike_angle: %s" % pos_angle)
    print("az: %s" % az)
    delta_angle = az - pos_angle
    print("delta: %s" % delta_angle)
    return np.sqrt(pos[0]**2 + pos[1]**2)*np.cos(delta_angle)

In [49]:
pos = [np.sqrt(3),1]
az = np.deg2rad(30)

beam_projection(pos,az)

mike_angle: 0.5235987755982989
az: 0.5235987755982988
delta: -1.1102230246251565e-16


1.9999999999999998